##Creating a datframe for the list of postal codes of Toronto


---



In [1]:
#importing the required libraries for the project

import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup

In [4]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
extracting_data = requests.get(url).text
wiki_data = BeautifulSoup(extracting_data, 'lxml')
print("WikiPage Scraped Successfully")

WikiPage Scraped Successfully


In [5]:
column_names = ['Postalcode','Borough','Neighborhood']
toronto = pd.DataFrame(columns = column_names)

content = wiki_data.find('div', class_='mw-parser-output')
table = content.table.tbody
postcode = 0
borough = 0
neighborhood = 0

for tr in table.find_all('tr'):
    i = 0
    for td in tr.find_all('td'):
        if i == 0:
            postcode = td.text
            i = i + 1
        elif i == 1:
            borough = td.text
            i = i + 1
        elif i == 2: 
            neighborhood = td.text.strip('\n').replace(']','')
    toronto = toronto.append({'Postalcode': postcode,
                              'Borough': borough,
                              'Neighborhood': neighborhood},ignore_index=True)

print('Data Collected Successfully')

Data Collected Successfully


In [8]:
toronto = toronto[toronto.Borough!='Not assigned']
toronto = toronto[toronto.Borough!= 0]
toronto.reset_index(drop = True, inplace = True)
i = 0
for i in range(0,toronto.shape[0]):
    if toronto.iloc[i][2] == 'Not assigned':
        toronto.iloc[i][2] = toronto.iloc[i][1]
        i = i+1



In [10]:
neighbors = toronto.groupby(['Postalcode','Borough'])['Neighborhood'].apply(', '.join).reset_index()
neighbors.head()

,Postalcode,Borough,Neighborhood
0,M1A\n,Not assigned\n,Not assigned\n
1,M1B\n,Scarborough\n,"Malvern, Rouge"
2,M1C\n,Scarborough\n,"Rouge Hill, Port Union, Highland Creek"
3,M1E\n,Scarborough\n,"Guildwood, Morningside, West Hill"
4,M1G\n,Scarborough\n,Woburn


In [13]:
neighbors["Borough"]=neighbors.Borough.str.replace('\\n','')
neighbors["Postalcode"]=neighbors.Postalcode.str.replace('\\n','')
neighbors["Neighborhood"]=neighbors.Neighborhood.str.replace('\\n','')

neighbors.head()

,Postalcode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M1B,Scarborough,"Malvern, Rouge"
2,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
3,M1E,Scarborough,"Guildwood, Morningside, West Hill"
4,M1G,Scarborough,Woburn


In [14]:
neighbors.drop(neighbors[neighbors['Borough']=='Not assigned'].index,axis=0,inplace=True)
neighbors.drop(neighbors[neighbors['Neighborhood']=='Not assigned'].index,axis=0,inplace=True)
neighbors.reset_index(drop=True,inplace=True)

neighbors.head()


,Postalcode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [15]:
neighbors.to_csv('Toronto.csv',index=False)

In [16]:
neighbors.shape

(103, 3)